**Requirement Gathering**

In [ ]:
!pip install python-dotenv
!pip install langchain langchain-openai openai
!pip install langchain_community
!pip install gradio
!pip install groq
!pip install numpy
!pip install soundfile
!pip install gTTS

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 38.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 MB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.6/322.6 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 70.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.4/127.4 kB 4.5 MB/s eta 0:00:00


In [ ]:
import os
import requests
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from bs4 import BeautifulSoup
from openai import OpenAI

In [ ]:
from google.colab import userdata
import openai
openai.api_key = userdata.get("OPENAI_API_KEY") # This sets the key within the openai library
load_dotenv(override=True)
MODEL = 'gpt-4o'


In [ ]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [ ]:
def get_company_core_values(company_name):
  response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "user", "content": f"what are core values of {company_name}"}
        ],
        temperature=0.1,
    )
  result = response.choices[0].message.content
  return result

In [ ]:
def get_role_summary(job_link):
    splunk_job = Website(job_link)
    splunk_job_web_content = splunk_job.get_contents()

    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": "You are an AI assistant specialized in extracting key job details from web content. Given the job posting, you will analyze and extract the 'Role Summary' section concisely. Ensure that the summary captures the job's main responsibilities and expectations of the role. Respond in markdown"},
            {"role": "user", "content": splunk_job_web_content + "\n\nCan you extract the role summary from this text?"}
        ],
        temperature=0.7,
    )

    return response.choices[0].message.content


In [ ]:
def generate_interviewer_intro_system_prompt(job_url):
    role_summary = get_role_summary(job_url)

    if not role_summary:  # Fallback check
        role_summary = "This role focuses on frontend development, collaborating with cross-functional teams, and contributing to building user-friendly web applications."

    system_prompt = (
        "You are a an AI assistant hiring manager conducting a behavioral interview. "
        "Your role is to set a welcoming, professional tone and introduce yourself with your name as Alex in a friendly manner."
        f"Create a friendly introduction for a mock interview as if you are the hiring manager. "
        f"Base it on this job summary:\n\n{role_summary}\n\n"
        "Your introduction should make the candidate feel comfortable while briefly mentioning the role and what your team does."
        "Also ask the candidate introducing him/her"
    )
    return system_prompt

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain
from langchain.schema import SystemMessage
from google.colab import userdata

api_key = userdata.get('OPENAI_API_KEY')
api_key_groq = userdata.get('GROQ_API')

# Initialize the Chat Model
llm = ChatOpenAI(model="gpt-4", openai_api_key=api_key)

# Memory to remember past conversations
memory = ConversationBufferMemory()

# Conversation Chain
conversation = ConversationChain(
    llm=llm,
    memory=memory
)

# Define user input
#user_input = "What is LangChain and how does it work?"

# Generate response
#response = conversation.predict(input=user_input)



<ipython-input-8-29f12539dee3>:11: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model="gpt-4", openai_api_key=api_key)
<ipython-input-8-29f12539dee3>:14: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory()
<ipython-input-8-29f12539dee3>:17: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :meth:`~RunnableWithMessageHistory: https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html` instead.
  conversation = Conversatio

In [ ]:
'''
import time
import tempfile
from gtts import gTTS

def text_to_speech(text):
    """Converts input text to speech using gTTS and saves it as an MP3 file."""

    if not text.strip():
        raise ValueError("Input text cannot be empty.")  # Handle empty input gracefully

    tts = gTTS(text=text, lang="en", tld="co.in")  # IN English accent

    # Save to a temporary file
    with tempfile.NamedTemporaryFile(suffix=".mp3", delete=False) as temp_file:
        temp_file_path = temp_file.name
        tts.save(temp_file_path)  # Save the generated speech

    return temp_file_path  # Returns the path to the generated audio file

# Example Usage & Timing
#start_time = time.time()
#audio_path = text_to_speech("Hello, my name is Alex. I work at Splunk as a Manager for the Engineering Team. Today, I’ll be conducting your behavioral interview for this role.")
#end_time = time.time()

#print(f"Audio file saved at: {audio_path}")
#print(f"Method execution time: {end_time - start_time} seconds")
'''

'\nimport time\nimport tempfile\nfrom gtts import gTTS\n\ndef text_to_speech(text):\n    """Converts input text to speech using gTTS and saves it as an MP3 file."""\n\n    if not text.strip():\n        raise ValueError("Input text cannot be empty.")  # Handle empty input gracefully\n\n    tts = gTTS(text=text, lang="en", tld="co.in")  # IN English accent\n\n    # Save to a temporary file\n    with tempfile.NamedTemporaryFile(suffix=".mp3", delete=False) as temp_file:\n        temp_file_path = temp_file.name\n        tts.save(temp_file_path)  # Save the generated speech\n\n    return temp_file_path  # Returns the path to the generated audio file\n\n# Example Usage & Timing\n#start_time = time.time()\n#audio_path = text_to_speech("Hello, my name is Alex. I work at Splunk as a Manager for the Engineering Team. Today, I’ll be conducting your behavioral interview for this role.")\n#end_time = time.time()\n\n#print(f"Audio file saved at: {audio_path}")\n#print(f"Method execution time: {end_t

In [ ]:
import openai
import tempfile

def text_to_speech(text):
    """Converts input text to speech using OpenAI's TTS API and saves it as an MP3 file."""

    if not text.strip():
        raise ValueError("Input text cannot be empty.")  # Handle empty input gracefully

    # Generate speech using OpenAI's TTS API
    response = openai.audio.speech.create(
        model="tts-1",
        voice="fable",
        input=text
    )

  # Save to a temporary file
    with tempfile.NamedTemporaryFile(suffix=".mp3", delete=False) as temp_file:
        temp_file_path = temp_file.name
        response.stream_to_file(temp_file_path)

    return temp_file_path  # Returns the path to the generated audio file

# Example Usage

In [ ]:
global current
current = ""
# Start conversation
def start_interview(input_job_url):

    """Starts the AI interview using the system prompt."""
    try:
        # AI introduces itself
        response = conversation.invoke(generate_interviewer_intro_system_prompt(input_job_url))
        bot_response = response['response']
        current = bot_response
        print("Bot:", bot_response)

        return text_to_speech(bot_response), bot_response

    except Exception as error:
        print("Error:", error)
        raise gr.Error("An error occurred while generating speech. Please check your Groq API key and try again.")
#start_interview()


In [ ]:
# Generate next question
def ask_next_question():
    """Ask the next question dynamically."""
    try:
        response = conversation.invoke("Can you ask me the next question?")
        bot_response = response['response']
        current = bot_response
        print("Bot:", bot_response)

        return text_to_speech(bot_response), bot_response

    except Exception as error:
        print("Error:", error)
        raise gr.Error("An error occurred while generating speech. Please check your API key and try again.")


In [ ]:
import gradio as gr
import groq
import io
import numpy as np
import soundfile as sf

In [ ]:
def transcribe_audio(audio):
    if audio is None:
        return ""

    client = groq.Client(api_key=api_key_groq)

    # Convert audio to the format expected by the model
    # The model supports mp3, mp4, mpeg, mpga, m4a, wav, and webm file types
    audio_data = audio[1]  # Get the numpy array from the tuple
    buffer = io.BytesIO()
    sf.write(buffer, audio_data, audio[0], format='wav')
    buffer.seek(0)

    bytes_audio = io.BytesIO()
    np.save(bytes_audio, audio_data)
    bytes_audio.seek(0)

    try:
        # Use Distil-Whisper English powered by Groq for transcription
        completion = client.audio.transcriptions.create(
            model="distil-whisper-large-v3-en",
            file=("audio.wav", buffer),
            response_format="text"
        )
        return completion
    except Exception as e:
        return f"Error in transcription: {str(e)}"

In [ ]:
evaluation_responses_list = []

In [ ]:
# Function to process and transcribe audio
def process_audio(audio):
    """Transcribes user audio and generates AI response."""

    if audio is None:
        return "No audio provided", "Please speak and try again."

    transcription = transcribe_audio(audio)
    evaluation_response = conversation.invoke(f"{transcription} is the response given by candidate for the answer. The evalution criteria should exactly match to what is asked for, technologies should match. If there is no match then grade the answer low. Please strictly evaluate the response based on the following criteria like Content Relevance(Does the response directly address the question? Is it factually correct? also rate on scale of 10) Answer Quality (How well-structured and complete is the answer? also rate on scale of 10), Confidence Level (Does the candidate express certainty in their response? also rate on scale of 10) Clarity & Communication (Is the response well-articulated and easy to understand? also rate on scale of 10) Depth of Knowledge (Does the answer show deep industry expertise or only surface-level understanding? also rate on scale of 10) Overall Score (Rate the response on a scale of 1-10, with a brief justification.) Please write response evaluation in markdown, including Content Relevance, Answer Quality, Confidence Level, Clarity & Communication, Depth of Knowledge, Overall Score out of 10.\n")
    evaluation_responses_list.append(evaluation_response)
    if not transcription or "Error" in transcription:
        return transcription, "Could not generate a response."
    ai_response = "Simulated AI response based on transcription."  # Replace with real AI generation

    return transcription, ai_response

In [ ]:
def fetch_company_info(input_company_name, input_job_url):
    global company_name
    global job_url
    company_name = input_company_name
    job_url = input_job_url
    #start_interview()
    #print(company_name, job_url)




In [ ]:
def combined_function():
    """Calls both functions in sequence."""
    company_info = fetch_company_info()  # Step 1: Fetch company info
    text_response, audio_response = start_interview()  # Step 2: Start interview
    return f"{company_info}\n\n{text_response}", audio_response

In [ ]:
# Gradio UI
js_func = """
function refresh() {
    const url = new URL(window.location);

    if (url.searchParams.get('__theme') !== 'dark') {
        url.searchParams.set('__theme', 'dark');
        window.location.href = url.href;
    }
}
"""

with gr.Blocks(js=js_func, theme=gr.themes.Ocean()) as demo:
    gr.Markdown("# <center> AI Interviewer </center>")

    with gr.Row():
      #company_name = gr.Textbox(label="Company Name")
      job_link = gr.Textbox(label="Job Link")

    # Create a submit button
   # submit_btn = gr.Button("Submit")
   # submit_btn.click(fn= fetch_company_info, inputs=[company_name, job_link])

    start_btn = gr.Button("Start Interview")
    output_audio = gr.Audio(label="Bot Response", autoplay= True)
    bot_response_text = gr.Textbox(label="Bot Response Text")

    start_btn.click(fn=start_interview, inputs=[job_link], outputs=[output_audio, bot_response_text])

    # 🎙️ Speech Input Section
    with gr.Row():
        audio_input = gr.Audio(sources=["microphone"],label="🎤 Speak Here!", type="numpy")

    # 📜 Transcription & Response Section
    with gr.Row():
        transcription_output = gr.Textbox(label="Your Response", inputs=audio_input)

    #submit_button = gr.Button("Process Speech", variant="primary")
    next_btn = gr.Button("Next Question")
    next_btn.click(fn=ask_next_question, inputs=[], outputs=[output_audio,bot_response_text])

    submit_button_eval = gr.Button("Evaluate", variant="primary")
    evaluation_output = gr.Textbox(label="Evaluation")
    langchain_output = gr.Textbox(label="Langchain")
    submit_button_eval.click(lambda: "\n".join([str(item['response']) for item in evaluation_responses_list]), inputs=[], outputs=evaluation_output)


    # 🔄 Function Connections
    audio_input.change(
        process_audio,
        inputs=[audio_input],
        outputs=[transcription_output]
    )
   # submit_button.click
    #    process_audio,
     #   inputs=[audio_input],
     #   outputs=[transcription_output]
    #)

    submit_button_eval.click(
        lambda: "\n".join([str(item['response']) for item in evaluation_responses_list]),
        inputs=[],
        outputs=evaluation_output
    )

    gr.Markdown("""
    ## How to use this app:
    1. Click **"Start Interview"** to receive a behavioral interview question.
    2. Click on the **microphone** and **speak your response**.
    3. Click **"Next Qusetion"** to generate next question.
    4. Click on Evaluate button for evaluation.
    """)

# Launch Gradio App
#server_name = "0.0.0.0"  # Change if needed
#demo.launch(server_name=server_name)

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://0bb61aedfeb8e01894.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
# Gradio UI
js_func = """
function refresh() {
    const url = new URL(window.location);

    if (url.searchParams.get('__theme') !== 'dark') {
        url.searchParams.set('__theme', 'dark');
        window.location.href = url.href;
    }
}
"""

with gr.Blocks(js=js_func, theme=gr.themes.Ocean()) as demo:
    gr.Markdown("# <center> AI Interviewer </center>")

    with gr.Row():
      #company_name = gr.Textbox(label="Company Name")
      job_link = gr.Textbox(label="Job Link")

    # Create a submit button
   # submit_btn = gr.Button("Submit")
   # submit_btn.click(fn= fetch_company_info, inputs=[company_name, job_link])

    start_btn = gr.Button("Start Interview")
    output_audio = gr.Audio(label="Bot Response", autoplay= True)
    bot_response_text = gr.Textbox(label="Bot Response Text")

    start_btn.click(fn=start_interview, inputs=[job_link], outputs=[output_audio, bot_response_text])

    # 🎙️ Speech Input Section
    with gr.Row():
        audio_input = gr.Audio(sources=["microphone"],label="🎤 Speak Here!", type="numpy")

    # 📜 Transcription & Response Section
    with gr.Row():
        transcription_output = gr.Textbox(label="Your Response", inputs=audio_input)

    #submit_button = gr.Button("Process Speech", variant="primary")
    next_btn = gr.Button("Next Question")
    next_btn.click(fn=ask_next_question, inputs=[], outputs=[output_audio,bot_response_text])

    submit_button_eval = gr.Button("Evaluate", variant="primary")
    evaluation_output = gr.Textbox(label="Evaluation")
    submit_button_eval.click(lambda: "\n".join([str(item['response']) for item in evaluation_responses_list]), inputs=[], outputs=evaluation_output)


    # 🔄 Function Connections
    audio_input.change(
        process_audio,
        inputs=[audio_input],
        outputs=[transcription_output]
    )
   # submit_button.click
    #    process_audio,
     #   inputs=[audio_input],
     #   outputs=[transcription_output]
    #)

    submit_button_eval.click(
        lambda: "\n".join([str(item['response']) for item in evaluation_responses_list]),
        inputs=[],
        outputs=evaluation_output
    )

    gr.Markdown("""
    ## How to use this app:
    1. Click **"Start Interview"** to receive a behavioral interview question.
    2. Click on the **microphone** and **speak your response**.
    3. Click **"Next Qusetion"** to generate next question.
    4. Click on Evaluate button for evaluation.
    """)

# Launch Gradio App
#server_name = "0.0.0.0"  # Change if needed
#demo.launch(server_name=server_name)

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://819b172429a6bd0cb7.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
import gradio as gr

js_func = """
function refresh() {
    const url = new URL(window.location);

    if (url.searchParams.get('__theme') !== 'dark') {
        url.searchParams.set('__theme', 'dark');
        window.location.href = url.href;
    }
}
"""

with gr.Blocks(js=js_func, theme=gr.themes.Ocean()) as demo:
    gr.Markdown("# <center> AI Interviewer </center>")

    with gr.Row():
        with gr.Column():  # Left side
            job_link = gr.Textbox(label="Job Link")

        with gr.Column():  # Right side - buttons stacked vertically
            start_btn = gr.Button("Start Interview", variant="primary")
            next_btn = gr.Button("Next Question")


    #start_btn = gr.Button("Start Interview")
    output_audio = gr.Audio(label="Bot Response", autoplay=True)
    bot_response_text = gr.Textbox(label="Bot Response Text")

    start_btn.click(fn=start_interview, inputs=[job_link], outputs=[output_audio, bot_response_text])

    with gr.Row():
        with gr.Column():  # Left Side - Speech and Transcription
            audio_input = gr.Audio(sources=["microphone"], label="🎤 Speak Here!", type="numpy")
            transcription_output = gr.Textbox(label="Your Response")  # Removed incorrect 'inputs' argument
            next_btn = gr.Button("Next Question")
            next_btn.click(fn=ask_next_question, inputs=[], outputs=[output_audio, bot_response_text])

        with gr.Column():  # Right Side - Evaluation
            submit_button_eval = gr.Button("Evaluate", variant="primary")
            evaluation_output = gr.Textbox(label="Evaluation")

    # Auto transcription when speech is recorded
    audio_input.change(
        process_audio,
        inputs=[audio_input],
        outputs=[transcription_output]
    )

    # Evaluation logic
    submit_button_eval.click(
        lambda: "\n".join([str(item['response']) for item in evaluation_responses_list]),
        inputs=[],
        outputs=evaluation_output
    )

    gr.Markdown("""
    ## How to use this app:
    1. Click **"Generate Question"** to receive a behavioral interview question.
    2. Click on the **microphone** and **speak your response**.
    3. Click **"Next Question"** to generate the next question.
    4. Click **"Evaluate"** to receive feedback on your answer.
    """)

# Launch Gradio App
server_name = "0.0.0.0"  # Change if needed
#demo.launch(server_name=server_name)


In [ ]:
import gradio as gr

js_func = """
function refresh() {
    const url = new URL(window.location);

    if (url.searchParams.get('__theme') !== 'dark') {
        url.searchParams.set('__theme', 'dark');
        window.location.href = url.href;
    }
}
"""
with gr.Blocks(js=js_func, theme=gr.themes.Ocean()) as demo:
    gr.Markdown("# <center> AI Interviewer </center>")

    with gr.Row():
        with gr.Column():  # Left side
            job_link = gr.Textbox(label="Job Link")
            #company_name = gr.Textbox(label="Company Name")

        with gr.Column():  # Right side - buttons stacked vertically
            start_btn = gr.Button("Start Interview", variant="primary")
            next_btn = gr.Button("Next Question")

    output_audio = gr.Audio(label="Bot Response", autoplay=True)
    bot_response_text = gr.Textbox(label="Bot Response Text")

    start_btn.click(fn=start_interview, inputs=[job_link], outputs=[output_audio, bot_response_text])

    with gr.Row():
        with gr.Column():  # Left Side - Speech and Transcription
            audio_input = gr.Audio(sources=["microphone"], label="🎤 Speak Here!", type="numpy")
            transcription_output = gr.Textbox(label="Your Response")
            next_btn.click(fn=ask_next_question, inputs=[], outputs=[output_audio, bot_response_text])

        with gr.Column():  # Right Side - Evaluation
            submit_button_eval = gr.Button("Evaluate", variant="primary")
            evaluation_output = gr.Textbox(label="Evaluation")

    # Auto transcription when speech is recorded
    audio_input.change(
        fn=process_audio,
        inputs=[audio_input],
        outputs=[transcription_output]
    )

    # Evaluation logic
    submit_button_eval.click(
        fn=lambda: "\n".join([str(item['response']) for item in evaluation_responses_list]),
        inputs=[],
        outputs=evaluation_output
    )

    gr.Markdown("""
    ## How to use this app:
    1. Click **"Start Interview"** to begin the interview.
    2. Click on the **microphone** and **speak your response**.
    3. Click **"Next Question"** to generate the next question.
    4. Click **"Evaluate"** to receive feedback on your answer.
    """)

# Launch Gradio App
demo.launch(server_name="0.0.0.0")


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ccb9b96693378f75ea.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
